In [1]:
# face detection with mtcnn on a photograph
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN

Using TensorFlow backend.


In [2]:
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense,Dropout
from keras.models import Model, load_model
#from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import tensorflow.keras.models as models

Using TensorFlow backend.


In [12]:
import tensorflow as tf

import keras as k
print(k.__version__)

2.3.0


In [13]:
print(tf.__version__)

2.0.0


In [14]:
m = models.load_model("face_mask_model.h5")

In [23]:
model = Sequential([
    Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    
    Conv2D(100, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dropout(0.5),
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [29]:
TRAINING_DIR = "./train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=64, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))

Found 1315 images belonging to 2 classes.
Found 194 images belonging to 2 classes.


In [30]:
train_generator

In [26]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

In [31]:
history = model.fit_generator(train_generator,
                              epochs=1,
                              validation_data=validation_generator)

Epoch 1/1
19/21 [==========================>...] - ETA: 18s - loss: 0.5304 - acc: 0.7321

OSError: cannot identify image file <_io.BytesIO object at 0x0000019F01328938>

In [2]:
def draw_image_with_boxes(pixels, bboxes):
	# load the image
    for box in bboxes:
        x, y, width, height = box
        x2, y2 = x + width, y + height
        # draw a rectangle over the pixels
        rectangle(pixels, (x, y), (x2, y2), (0,0,255), 1)

In [3]:
# plot photo with detected faces using opencv cascade classifier
from cv2 import imread
from cv2 import imshow
from cv2 import waitKey
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
from cv2 import rectangle
# load the photograph
pixels = imread('test1.jpg')
# load the pre-trained model
classifier = CascadeClassifier('haarcascade_frontalface_default.xml')
# perform face detection
bboxes = classifier.detectMultiScale(pixels)
# print bounding box for each detected face
draw_image_with_boxes(pixels,bboxes)
# show the image
imshow('face detection', pixels)
# keep the window open until we press a key
waitKey(0)
# close the window
destroyAllWindows()

In [3]:
import numpy as np
import cv2
from cv2 import imread
from cv2 import imshow
from cv2 import waitKey
from cv2 import destroyAllWindows
from cv2 import CascadeClassifier
from cv2 import rectangle
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
m = models.load_model("face_mask_model.h5")

cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    classifier = CascadeClassifier('haarcascade_frontalface_default.xml')
    # perform face detection
    bboxes = classifier.detectMultiScale(frame)
    
    # print bounding box for each detected face
    #draw_image_with_boxes(frame,bboxes)
    for box in bboxes:
        x, y, width, height = box
        x2, y2 = x + width, y + height
        # draw a rectangle over the pixels
        face = frame[y:y2, x:x2]
        face = cv2.resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        (mask, withoutMask) = m.predict(face)[0]
        label = "Mask" if mask > withoutMask else "No Mask"
        color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
        # include the probability in the label
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        # display the label and bounding box rectangle on the output
        # frame
        cv2.putText(frame, label, (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        #cv2.rectangle(image, (startX, startY), (endX, endY), color, 2)
        rectangle(frame, (x, y), (x2, y2), (0,0,255), 1)

    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [65]:
frame[y:y2, x:x2]

array([[[  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255],
        ...,
        [  0,   0, 255],
        [  0,   0, 255],
        [  0,   0, 255]],

       [[  0,   0, 255],
        [ 87,  90,  96],
        [ 56,  59,  66],
        ...,
        [ 21,  17,  22],
        [ 22,  19,  23],
        [ 23,  20,  25]],

       [[  0,   0, 255],
        [ 68,  74,  80],
        [ 51,  56,  62],
        ...,
        [ 21,  17,  22],
        [ 22,  19,  23],
        [ 23,  20,  23]],

       ...,

       [[  0,   0, 255],
        [ 94,  91, 105],
        [ 95,  92, 106],
        ...,
        [ 92, 110, 129],
        [ 70,  88, 107],
        [ 57,  64,  82]],

       [[  0,   0, 255],
        [ 94,  91, 107],
        [ 94,  91, 107],
        ...,
        [ 87, 107, 130],
        [ 91, 111, 134],
        [ 80,  90, 107]],

       [[  0,   0, 255],
        [ 92,  88, 106],
        [ 92,  88, 106],
        ...,
        [ 82, 101, 126],
        [ 91, 110, 136],
        [ 90, 104, 123]]

In [13]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)
detector = MTCNN()

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR)
    # detect faces in the image
    faces = detector.detect_faces(frame)
    x, y, width, height = faces[0]['box']
    
    cv2.rectangle(frame, (x, faces[0]['keypoints']['nose'][1]), (x + width, y + height), (255,0,0), 2)
    

    # Display the resulting frame
    #cv2.rectangle(frame,(x,y),(height,width),(0,255,0),3)
    cv2.imshow('frame',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

IndexError: list index out of range

In [25]:
faces = detector.detect_faces(pixels)


In [8]:
faces

[{'box': [146, 99, 168, 223],
  'confidence': 0.9999768733978271,
  'keypoints': {'left_eye': (190, 175),
   'right_eye': (268, 181),
   'nose': (223, 219),
   'mouth_left': (187, 264),
   'mouth_right': (256, 269)}}]

In [12]:
faces[0]['keypoints']['nose'][1]

(223, 219)